In [15]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# Load the image from the file
img = Image.open("041224A1.jpeg")

# Use pytesseract to do optical character recognition on the image
temp_text = pytesseract.image_to_string(img, lang='eng')
# Print the extracted text
temp_text


'Map Show More\n\nBurlington\n\n2 HUMIDITY\n\n80%\n\n'

In [10]:
import re
# Use a regular expression to find the temperature value
match = re.search(r'@ (\d+)°', temp_text)
if match:
    temp = match.group(1)
    print(f"The detected temperature is: {temp}°")
else:
    print("Temperature not found in the text.")
    

The detected temperature is: 34°


In [17]:
import tkinter as tk
from tkinter import filedialog
import pytesseract
from PIL import Image
import re
import os
import pandas as pd

# Setting the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def process_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='eng', config='--psm 6')
    return text

def extract_temperature_and_humidity(text):
    temperature = None
    humidity = None

    # Regex to match both temperature and humidity
    temp_match = re.search(r'(?:@ |a: )(\d+)°', text)
    humidity_match = re.search(r'HUMIDITY\n\n(\d+)%', text)

    if temp_match:
        temperature = temp_match.group(1)
    if humidity_match:
        humidity = humidity_match.group(1)

    return temperature, humidity

# DataFrame to store all data cumulatively
cumulative_data_df = pd.DataFrame(columns=['Image', 'Temperature', 'Humidity'])

def upload_action():
    file_paths = filedialog.askopenfilenames()
    global cumulative_data_df
    for path in file_paths:
        text = process_image(path)
        temperature, humidity = extract_temperature_and_humidity(text)
        data = {'Image': os.path.basename(path)}
        if temperature:
            data['Temperature'] = f"{temperature}°"
        if humidity:
            data['Humidity'] = f"{humidity}%"
        cumulative_data_df = cumulative_data_df.append(data, ignore_index=True)
    
    result_text.delete('1.0', tk.END)  # Clear existing text
    result_text.insert(tk.END, cumulative_data_df.to_string(index=False))  # Display dataframe as string in text widget

def save_csv():
    save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
    if save_path:
        global cumulative_data_df
        cumulative_data_df.to_csv(save_path, index=False)
        print(f"Data saved to {save_path}")
        cumulative_data_df = pd.DataFrame(columns=['Image', 'Temperature', 'Humidity'])  # Reset DataFrame after saving

root = tk.Tk()
root.title("Temperature and Humidity Extractor")

# Button to upload images
upload_btn = tk.Button(root, text="Upload Images", command=upload_action)
upload_btn.pack()

# Button to save CSV
save_btn = tk.Button(root, text="Save CSV", command=save_csv)
save_btn.pack()

# Text widget to display results
result_text = tk.Text(root, height=20, width=80)
result_text.pack()

root.mainloop()


C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative_data_df = cumulative_data_df.append(data, ignore_index=True)
C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative_data_df = cumulative_data_df.append(data, ignore_index=True)
C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative_data_df = cumulative_data_df.append(data, ignore_index=True)
C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative_data_df = cumulative_data_df.append(data, ignore_index=True)
C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative_data_df = cumulative_data_df.append(data, ignore_index=True)
C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative_data_df = cumulative_data_df.append(data, ignore_index=True)
C:\Users\msikh\AppData\Local\Temp\ipykernel_13012\3837233154.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

Data saved to C:/Users/msikh/EV_Project/EV_Discharging_Data/NEUTC/EV Trip Data/Temp_Data.csv


In [33]:
import pandas as pd
import re

# Load the CSV file
# Load the CSV file with a specified encoding
file_path = r'C:\Users\msikh\EV_Project\EV_Discharging_Data\NEUTC\EV Trip Data\EV_Temp_Data.csv'  # Replace with actual file path

# Try with different encodings if one doesn't work
try:
    Temp_Data_EV = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        Temp_Data_EV = pd.read_csv(file_path, encoding='latin1')
    except UnicodeDecodeError:
        Temp_Data_EV = pd.read_csv(file_path, encoding='iso-8859-1')

def extract_date_and_letter(trip_name):
    match = re.match(r"(\d{2})(\d{2})(\d{2})([A-Z])", trip_name)
    if match:
        day, month, year, letter = match.groups()
        date = f"20{year}-{month}-{day}"
        return date, letter
    return trip_name, ''

# Process the 'Trip Name' column
if 'Image' in Temp_Data_EV.columns:
    Temp_Data_EV['Date'], Temp_Data_EV['Letter'] = zip(*Temp_Data_EV['Image'].apply(extract_date_and_letter))
    Temp_Data_EV.drop(columns=['Image'], inplace=True)
Temp_Data_EV

,Temperature,Date,Letter
0,34°,2024-01-04,B
1,29°,2024-02-04,A
2,36°,2024-03-04,A
3,41°,2024-03-04,B
4,31°,2024-04-04,A
5,62°,2024-09-04,A
6,37°,2024-09-04,B
7,47°,2024-11-04,A
8,45°,2024-11-04,B
9,61°,2024-12-04,A


In [38]:
def update_temperature(df, date, letter, temperature):
    # Remove the unit and handle the case where temperature is Null
    if pd.isna(temperature) or temperature is None or temperature == "Null":
        temperature_val = None
    else:
        temperature_val = int(temperature.replace('°', ''))
    
    # Check if the entry exists
    mask = (df['Date'] == date) & (df['Letter'] == letter)
    if not df[mask].empty:
        # Update existing entry
        df.loc[mask, 'Temperature'] = temperature_val
    else:
        # Append new entry
        new_row = pd.DataFrame({'Temperature': [temperature_val], 'Date': [date], 'Letter': [letter]})
        df = pd.concat([df, new_row], ignore_index=True)

    return df

# Update the DataFrame with new data
for index, row in Temp_Data_EV.iterrows():
    df = update_temperature(df, row['Date'], row['Letter'], row['Temperature'])

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path_2, index=False, encoding='utf-8')

In [43]:
# Load the CSV file

df = pd.read_csv(file_path_2)

# Normalize the date format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%Y-%d-%m').fillna(
    pd.to_datetime(df['Date'], errors='coerce', format='%d/%m/%Y')
)

# Remove the unit from temperature and convert to numeric
df['Temperature'] = df['Temperature'].replace('°', '', regex=True).astype(float)



In [44]:
df

,Unnamed: 0,Too long,Date,Letter,Time Summary_GPS,Time Summary_Apple,Avg Altitude (ft),Avg Speed_GPS,Avg Speed_Apple,Distance_GPS,Distance_Apple,Elevation Gain (ft),Temperature
0,0.0,0.0,2024-02-26,B,01h08m09s,1:06:46,930,47.5,48.2,53.9,53.65,"2,996",31.0
1,1.0,0.0,2024-02-27,A,0 1h03m28s,1:03:20,939,54.0,53.5,57.5,56.52,"3,149",47.0
2,2.0,0.0,2024-02-29,A,01h08m19s,1:08:02,918,47.6,47.6,54.2,54.04,"2,847",16.0
3,3.0,0.0,2024-03-12,B,01h05m09s,1:05:43,946,49.5,49.3,53.8,54.03,"2,968",NaN
4,4.0,0.0,2024-03-14,A,01h06m52s,1:06:38,941,48.5,48.6,54.0,54.00,"2,900",49.0
5,5.0,0.0,2024-03-14,B,01h09m53s,1:09:41,937,46.3,46.1,53.9,53.57,"2,961",34.0
6,6.0,0.0,2024-03-15,A,01h40m59s,1:10:14,944,32.1,46.1,54.0,54.05,"2,766",40.0
7,7.0,1.0,2024-03-15,B,01h07m41s,1:07:24,932,50.3,50.5,56.7,56.82,"3,162",38.0
8,8.0,0.0,2024-03-19,A,01h04m58s,1:04:38,932,49.9,50.1,54.0,54.07,"2,819",33.0
9,9.0,1.0,2024-03-20,B,0 1h08m27s,1:08:44,951,49.7,49.5,56.8,56.74,"3,105",32.0


In [50]:
import datete
# Convert text to a list of dates
dates = df['Date']

# Define a function to parse dates and standardize format
def parse_date(date_str):
    for fmt in ("%Y-%d-%m", "%d/%m/%Y"):
        try:
            return datetime.strptime(date_str, fmt).strftime("%Y-%d-%m")
        except ValueError:
            continue
    raise ValueError(f"Date format not recognized: {date_str}")

# Apply the function to all dates
standardized_dates = [parse_date(date) for date in dates]

df["Date"]=standardized_dates



df

AttributeError: module 'datetime' has no attribute 'strptime'

In [47]:
df

,Unnamed: 0,Too long,Date,Letter,Time Summary_GPS,Time Summary_Apple,Avg Altitude (ft),Avg Speed_GPS,Avg Speed_Apple,Distance_GPS,Distance_Apple,Elevation Gain (ft),Temperature
0,0.0,0.0,2024-02-26,B,01h08m09s,1:06:46,930,47.5,48.2,53.9,53.65,"2,996",31.0
1,1.0,0.0,2024-02-27,A,0 1h03m28s,1:03:20,939,54.0,53.5,57.5,56.52,"3,149",47.0
2,2.0,0.0,2024-02-29,A,01h08m19s,1:08:02,918,47.6,47.6,54.2,54.04,"2,847",16.0
3,3.0,0.0,2024-03-12,B,01h05m09s,1:05:43,946,49.5,49.3,53.8,54.03,"2,968",NaN
4,4.0,0.0,2024-03-14,A,01h06m52s,1:06:38,941,48.5,48.6,54.0,54.00,"2,900",49.0
5,5.0,0.0,2024-03-14,B,01h09m53s,1:09:41,937,46.3,46.1,53.9,53.57,"2,961",34.0
6,6.0,0.0,2024-03-15,A,01h40m59s,1:10:14,944,32.1,46.1,54.0,54.05,"2,766",40.0
7,7.0,1.0,2024-03-15,B,01h07m41s,1:07:24,932,50.3,50.5,56.7,56.82,"3,162",38.0
8,8.0,0.0,2024-03-19,A,01h04m58s,1:04:38,932,49.9,50.1,54.0,54.07,"2,819",33.0
9,9.0,1.0,2024-03-20,B,0 1h08m27s,1:08:44,951,49.7,49.5,56.8,56.74,"3,105",32.0
